In [ ]:
import os
import glob
import cv2
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pycocotools.coco import COCO
import pandas as pd
import json

In [ ]:
def getAnnotations(id):
    '''
    Load annotations with segmentation mask and bounding box 
    Input: image id
    Output: 
    '''
    image = coco.loadImgs(id)
    anns_ids = coco.getAnnIds(id)
    
    filename = 'images/{}'.format(image[0]['file_name'])
    height, width = cv2.imread(filename).shape[:2]
    
    lesions = []
    for anns_id in anns_ids:
        anns = coco.loadAnns(anns_id)
        segs = anns[0]['segmentation']
        category = anns[0]['category_id']
        bbox = []
        for seg in segs:
            X = seg[0::2]
            Y = seg[1::2]
            bbox = [min(X), min(Y), max(X), max(Y)]
            lesion = {
                'category_id': category,
                'segmentation': [seg],
                'bbox': bbox,
            }
            lesions.append(lesion)
   
    annotation = {
        'file_name': filename,
        'image_id': id,
        'height': height,
        'width': width,
        'annotations': lesions
    }
    return annotation

def showAnns(anns):
    '''
    Display image and annotations with segs and bbox
    Input: processed annotation
    Output: figures
    '''
    filename = anns['file_name'].split('/')[1]
    image_path = './images/{}'.format(filename)
    lesions_path = {
        'exu': './lesions/exudates/{}'.format(filename),
        'hem': './lesions/hemorrhage/{}'.format(filename),
        'mas': './lesions/microaneurysms/{}'.format(filename)
    }
    print(filename)
    img = cv2.imread(image_path)
    h,w,_ = img.shape
    exu = cv2.imread(lesions_path['exu'], 0)
    if exu is None:
        exu = np.zeros((h,w))
    hem = cv2.imread(lesions_path['hem'], 0)
    if hem is None:
        hem = np.zeros((h,w))
    mas = cv2.imread(lesions_path['mas'], 0)
    if mas is None:
        mas = np.zeros((h,w))

    for lesion in anns['annotations']:
        if lesion == []:
            continue
        seg = lesion['segmentation'][0]
        X = seg[0::2]
        Y = seg[1::2]
        start_point = (int(min(X)), int(min(Y)))
        end_point = (int(max(X)), int(max(Y)))
        img = cv2.rectangle(img,start_point, end_point,(0,255,0),12)
        
    fig = plt.figure(figsize=(24,12))
    plt.subplot(111), plt.title('img'), plt.imshow(img[:,:,::-1])
    plt.subplot(142), plt.title('hemorrhage'), plt.imshow(hem, cmap='gray'),
    plt.subplot(143), plt.title('exudates'), plt.imshow(exu, cmap='gray'),
    plt.subplot(144), plt.title('microaneurysms'), plt.imshow(mas, cmap='gray'),

def loadAnns(root):
    '''
    Load CTEH-DR-lesions dataset with annotations: segmentations + bbox
    Input: dataset path
    Output: annotations
    '''
    filenames = [x for x in os.listdir('{}/images'.format(root)) if x[:4] == 'CTEH']
    anns = []

    for id, filename in tqdm(enumerate(filenames)):
        ann = getAnnotations(id+1)
        anns.append(ann)
    return anns
    

In [ ]:
# Main
root = '.'
json_path = 'instances_default.json'

coco = COCO(json_path)
anns = loadAnns(root)
train_set = anns[:160]
val_set = anns[160:]
# Save annotations into json files
with open('annotations.json', 'w') as annotations:
    json.dump(anns, annotations)
with open('train.json', 'w') as train:
    json.dump(train_set, train)
with open('val.json', 'w') as val:
    json.dump(val_set, val)